In [1]:
import aocd
import re
import numpy as np
%run helper.ipynb
puzzle = aocd.models.Puzzle(year=2024, day=9)
data = puzzle.input_data

In [2]:
class File(object):
    def __init__(self, start, length, ID):
        self.blocks = list(range(start, start+length))
        self.id = ID
    
    def shift_to(self, pos):
        self.blocks[self.blocks.index(max(self.blocks))] = pos
        
    def has_blocks_left(self, pos):
        return any([x > pos for x in self.blocks])
        
    def checksum(self):
        return sum(self.blocks) * self.id

    def __repr__(self):
        return f"{self.id}: {self.blocks}"

In [3]:
is_file = True
files = []
empty_spots = []
location = 0
# Build files
for v in data:
    v = int(v)
    if is_file:
        files.append(File(location, v, len(files)))
    else:
        empty_spots += list(range(location, location + v))
    is_file = not is_file
    location += v
    
used_files = []
insert_pos = 0
# Shift file blocks
while(len(files) > 0):
    while(files[-1].has_blocks_left(insert_pos)):
        if insert_pos not in empty_spots:
            insert_pos += 1
            continue
        files[-1].shift_to(insert_pos)
        insert_pos += 1
    used_files.append(files.pop())

In [4]:
puzzle.answer_a = sum([f.checksum() for f in used_files])

In [5]:
class Sector(object):
    def __init__(self, start, size, ID):
        self.start = start
        self.size = size
        self.id = ID
    
    def shift_to(self, other_sector):
        if other_sector.size < self.size:
            return (other_sector, False)
        self.start = other_sector.start
        other_sector.start += self.size
        other_sector.size -= self.size
        if other_sector.size == 0:
            return (None, True)
        return (other_sector, True)
    
        
    def checksum(self):
        return sum(range(self.start, self.start + self.size)) * self.id

    def __repr__(self):
        return f"{self.id}: {self.start}-{self.start+self.size-1}"

In [6]:
is_file = True
files = []
empty_spots = []
location = 0
# Build files
for v in data:
    v = int(v)
    if is_file:
        files.append(Sector(location, v, len(files)))
    else:
        empty_spots.append(Sector(location, v, 0))
    is_file = not is_file
    location += v

In [7]:
for file in files[::-1]:
    moved = False
    empty_spot_idx = 0
    while not moved and empty_spots[empty_spot_idx].start < file.start:
        empty_sector, moved = file.shift_to(empty_spots[empty_spot_idx])
        if empty_sector is None:
            empty_spots.pop(empty_spot_idx)
        else:
            empty_spot_idx += 1        

In [8]:
puzzle.answer_b = sum([f.checksum() for f in files])